<a href="https://colab.research.google.com/github/LawrenceLLY/GNN_Pun_Detection/blob/main/BERT_GNN_MTL_Control_BiLSTM_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install and Import Packages

In [ ]:
!pip install transformers
!pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-2.0.0+cu118.html
!pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-2.0.0+cu118.html
!pip install torch-cluster -f https://pytorch-geometric.com/whl/torch-2.0.0+cu118.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-2.0.0+cu118.html
# The same as Torch version and CUDA version (torch.__version__ is 2.0.0+cu118)
!pip install torch-geometric
!python3 -m spacy download en_core_web_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 41.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 19.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://pytorch-geometric.com/whl/torch-2.0.0+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 35.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://pytorch-geometric.com/whl/torch-2.0.0+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 39.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://pytorch-geometric.com/whl/tor

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
print(torch.__version__)
from torch_geometric.nn import GCNConv, global_mean_pool, global_max_pool
from torch_geometric.data import Data, DataLoader
# import numpy as np
from transformers import BertModel, BertTokenizer, BertConfig
import csv
import spacy
import networkx as nx
#from gensim.models import Word2Vec
from tqdm import tqdm
import xml.etree.ElementTree as ET
from nltk.corpus import wordnet as wn
from nltk.corpus.reader.wordnet import Synset
from nltk.wsd import lesk
import nltk
import pickle
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from google.colab import drive
drive.mount('/content/drive')

nltk.download('wordnet')
nltk.download('punkt')

2.0.0+cu118
Mounted at /content/drive


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
nlp = spacy.load('en_core_web_sm') # DO NOT split a sentence for more 1 time
# If spliting a sentence more than 1 time, the result may different from the same split
model_path = "/content/drive/My Drive/my_GNN_MTL_PT_model_v1.pt"  # Choose your desired path and filename
# Model parameters
bert_model_name = 'bert-base-uncased'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data Preprocessing

## Download the semeval2017_task7 dataset

In [ ]:
!wget https://alt.qcri.org/semeval2017/task7/data/uploads/semeval2017_task7.tar.xz
!tar -xf semeval2017_task7.tar.xz
#!tar -xvf semeval2017_task7.tar.xz
#%cd semeval2017_task7/
#%cd ..
%ls

--2023-05-04 04:19:18--  https://alt.qcri.org/semeval2017/task7/data/uploads/semeval2017_task7.tar.xz
Resolving alt.qcri.org (alt.qcri.org)... 80.76.166.231
Connecting to alt.qcri.org (alt.qcri.org)|80.76.166.231|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 748424 (731K) [application/x-xz]
Saving to: ‘semeval2017_task7.tar.xz.3’

semeval2017_task7.t 100%[===================>] 730.88K   717KB/s    in 1.0s    

2023-05-04 04:19:20 (717 KB/s) - ‘semeval2017_task7.tar.xz.3’ saved [748424/748424]

drive/              semeval2017_task7.tar.xz    semeval2017_task7.tar.xz.3
sample_data/        semeval2017_task7.tar.xz.1
semeval2017_task7/  semeval2017_task7.tar.xz.2


## homographic

In [ ]:
f = 'semeval2017_task7/data/test/subtask1-homographic-test.xml'

mytree = ET.parse(f)
myroot = mytree.getroot()

puns_hom = []
for item in myroot.findall('./text'):
    dict1 = {}
    dict1[item.attrib['id']] = {}
    for child in item:
        idd = child.attrib['id']
        dict1[item.attrib['id']][idd] = child.text
    for pun in dict1.values():
        puns_hom.append([pun[x].replace(u'\xa0', '_') for x in pun])

print(puns_hom[0])

['They', 'hid', 'from', 'the', 'gunman', 'in', 'a', 'sauna', 'where', 'they', 'could', 'sweat', 'it', 'out', '.']


In [ ]:
gold_hom = []
with open('semeval2017_task7/data/test/subtask1-homographic-test.gold', 'r') as fin:
    for row in fin:
        gold_hom.append(int(row.strip().split('\t')[1]))
print(gold_hom)

[1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 

In [ ]:
location_hom = [-10000 for _ in range(len(puns_hom))]
with open('semeval2017_task7/data/test/subtask2-homographic-test.gold', 'r') as fin:
    for row in fin:
        # The default is start from 1
        pun_index = int(row.strip().split('\t')[1].split('_')[2]) - 1
        location_hom[int(row.strip().split('\t')[1].split('_')[1]) - 1] = pun_index
print(location_hom)

[11, 8, 6, 4, 14, -10000, 13, 16, 10, 12, 8, -10000, -10000, 13, 14, 12, -10000, 7, 2, 16, 8, 14, 6, 6, 4, 14, 9, 4, 3, 7, 11, -10000, 3, 14, -10000, -10000, -10000, 8, 19, -10000, 15, -10000, 7, 11, 5, 8, 12, 9, -10000, 8, 7, 18, 7, -10000, 11, -10000, -10000, 14, -10000, 19, 9, 8, 9, -10000, -10000, -10000, 14, -10000, 10, 8, 20, 9, 8, 8, 12, 6, 13, 17, -10000, 4, 8, 5, 3, -10000, 14, 19, 12, 10, -10000, 2, 14, 9, 13, -10000, -10000, 11, 5, 14, 9, 19, -10000, -10000, 6, 16, -10000, 8, 16, 9, 9, 10, -10000, 6, 10, -10000, 10, 13, 4, -10000, 6, 11, -10000, -10000, -10000, 6, 16, 0, 15, 15, -10000, 10, -10000, 3, 11, -10000, 9, 14, 4, 14, 11, 11, 16, 16, -10000, 10, 10, -10000, 7, 14, 8, 7, 13, 20, 1, 8, -10000, 9, 7, -10000, 20, 8, 11, 7, 12, 11, 12, -10000, 5, -10000, 7, 22, -10000, 9, 7, 16, 7, -10000, 7, 14, 9, -10000, 16, -10000, 6, -10000, 11, 12, -10000, 21, 10, -10000, 10, -10000, 13, -10000, 6, 13, 12, 2, 20, -10000, 12, 11, 6, 16, -10000, 14, 7, 12, 16, -10000, -10000, -10000,

In [ ]:
assert len(puns_hom) == len(gold_hom)
assert len(gold_hom) == len(location_hom)

## heterographic

In [ ]:
f = 'semeval2017_task7/data/test/subtask1-heterographic-test.xml'

mytree = ET.parse(f)
myroot = mytree.getroot()

puns_het = []
for item in myroot.findall('./text'):
    dict1 = {}
    dict1[item.attrib['id']] = {}
    for child in item:
        idd = child.attrib['id']
        dict1[item.attrib['id']][idd] = child.text
    for pun in dict1.values():
        puns_het.append([pun[x].replace(u'\xa0', '_') for x in pun])

print(puns_het[0])

["'", "'", 'I', "'", 'm', 'halfway', 'up', 'a', 'mountain', ',', "'", "'", 'Tom', 'alleged', '.']


In [ ]:
gold_het = []
with open('semeval2017_task7/data/test/subtask1-heterographic-test.gold', 'r') as fin:
    for row in fin:
        gold_het.append(int(row.strip().split('\t')[1]))
print(gold_het)

[1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 

In [ ]:
location_het = [-10000 for _ in range(len(puns_het))]
with open('semeval2017_task7/data/test/subtask2-heterographic-test.gold', 'r') as fin:
    for row in fin:
        # The default is start from 1
        pun_index = int(row.strip().split('\t')[1].split('_')[2]) - 1
        location_het[int(row.strip().split('\t')[1].split('_')[1]) - 1] = pun_index
print(location_het)

[13, 12, -10000, 10, 4, -10000, 5, 3, 5, 7, 12, 11, 23, -10000, 2, 10, 6, -10000, 15, 15, -10000, 16, 12, 5, 6, 11, -10000, 9, -10000, -10000, 13, -10000, 23, 15, 13, 16, -10000, 12, 14, 5, -10000, 10, 9, 14, 7, 9, 7, -10000, 7, 3, 10, -10000, -10000, 28, 10, 6, -10000, 13, 12, 15, -10000, 12, -10000, 15, 17, 7, 6, -10000, 13, 26, 13, 3, 12, 12, 16, 11, 4, 6, -10000, 10, -10000, 10, 10, -10000, -10000, 9, 11, -10000, 10, 5, -10000, 7, -10000, 6, 18, 8, 3, -10000, 11, 6, 15, 5, -10000, 4, 3, 14, 7, 13, -10000, -10000, 7, 10, 12, -10000, -10000, 6, 30, -10000, 6, -10000, 14, 6, 14, 11, 6, 39, -10000, -10000, 11, -10000, 16, -10000, 8, 15, -10000, 24, 23, 17, -10000, 13, 5, 11, 14, 3, 13, -10000, -10000, 4, 6, 8, -10000, -10000, 12, 14, -10000, 1, 3, 6, 12, -10000, -10000, 4, -10000, -10000, 17, 25, -10000, 9, -10000, -10000, -10000, 19, -10000, 6, -10000, -10000, 6, -10000, 16, -10000, 22, -10000, 13, 8, 12, 19, 17, 16, 9, -10000, 14, -10000, 14, 5, 10, 12, 5, -10000, -10000, 16, 6, 6, 8

In [ ]:
assert len(puns_het) == len(gold_het)
assert len(gold_het) == len(location_het)

## Puns of the Day

In [ ]:
texts_PTD = []
labels_PTD = []
# nlp = spacy.load('en_core_web_sm')

# text = 'My first birthday was great. My 2. was even better.'
# sentences = [str(tok) for sent in nlp(text).sents for tok in sent]

# opening the CSV file
with open("/content/drive/My Drive/puns_pos_neg_data.csv", mode ='r') as file:

    # reading the CSV file
    csvFile = csv.reader(file)
    
    # displaying the contents of the CSV file
    for line in csvFile:
        #print(line)
        labels_PTD.append(0 if line[0] == "-1" else 1)
        # texts_PTD.append([str(tok) for sent in nlp(line[1]).sents for tok in sent])
        texts_PTD.append(line[1].split())

del texts_PTD[0] # delete the head
del labels_PTD[0] # delete the head
assert len(texts_PTD) == len(labels_PTD)
print(texts_PTD[0])
print(texts_PTD[1])

['i', 'used', 'to', 'be', 'a', 'banker', 'but', 'i', 'lost', 'interest']
['i', 'm', 'glad', 'i', 'know', 'sign', 'language', 'it', 's', 'pretty', 'handy']


## Get Total Dataset

### Using spaCy to correct the location index after segmentation.

In [ ]:
#new_puns_hom = []
#new_puns_het = []
new_location_hom = []
new_location_het = []

In [ ]:
for idx, senten_list in enumerate(puns_hom):
    new_sent = [str(tok) for sent in nlp(' '.join(senten_list)).sents for tok in sent]
    # new_puns_hom.append(new_sent)
    # DO NOT update! Because the dataset package will using the origial sentence to split again!
    # If split more than 1 times, the result may be different
    if gold_hom[idx] > 0.5:
        if new_sent[:location_hom[idx]+1] != senten_list[:location_hom[idx]+1]:
            print(senten_list)
            print(new_sent)
            print(senten_list[location_hom[idx]])
            print(location_hom[idx])
            count_add = 0
            for each_word in senten_list[:location_hom[idx]]:
                count_add += len(nlp(each_word))
            print(new_sent[count_add])
            print(count_add)
            new_location_hom.append(count_add)
        else:
            new_location_hom.append(location_hom[idx])
    else:
        new_location_hom.append(location_hom[idx]) 

['Don’t', 'let', 'adversity', 'get', 'you', 'down', 'unless', 'you', 'are', 'on', 'your', 'knees']
['Do', 'n’t', 'let', 'adversity', 'get', 'you', 'down', 'unless', 'you', 'are', 'on', 'your', 'knees']
down
5
down
6
['We’ve', 'got', 'children’s', 'church', 'because', 'we', 'know', 'how', 'to', 'kid', 'around']
['We', '’ve', 'got', 'children', '’s', 'church', 'because', 'we', 'know', 'how', 'to', 'kid', 'around']
kid
9
kid
11
['Don’t', 'let', 'your', 'footprints', 'in', 'the', 'sands', 'of', 'time', 'be', 'the', 'mark', 'of', 'a', 'heel']
['Do', 'n’t', 'let', 'your', 'footprints', 'in', 'the', 'sands', 'of', 'time', 'be', 'the', 'mark', 'of', 'a', 'heel']
heel
14
heel
15
['We', 'don’t', 'read', 'palms', ';', 'we', 'wave', 'them', '.']
['We', 'do', 'n’t', 'read', 'palms', ';', 'we', 'wave', 'them', '.']
palms
3
palms
4
['There’s', 'no', 'high', 'like', 'the', 'most', 'high', '.']
['There', '’s', 'no', 'high', 'like', 'the', 'most', 'high', '.']
high
2
high
3
['If', 'you’re', 'looking', '

In [ ]:
for idx, senten_list in enumerate(puns_het):
    new_sent = [str(tok) for sent in nlp(' '.join(senten_list)).sents for tok in sent]
    # new_puns_het.append(new_sent)
    # DO NOT update! Because the dataset package will using the origial sentence to split again!
    # If split more than 1 times, the result may be different
    if gold_het[idx] > 0.5:
        if new_sent[:location_het[idx]+1] != senten_list[:location_het[idx]+1]:
            print(senten_list)
            print(new_sent)
            print(senten_list[location_het[idx]])
            print(location_het[idx])
            count_add = 0
            for each_word in senten_list[:location_het[idx]]:
                count_add += len(nlp(each_word))
            print(new_sent[count_add])
            print(count_add)
            new_location_het.append(count_add)
        else:
            new_location_het.append(location_het[idx])
    else:
        new_location_het.append(location_het[idx])       

["'", 'Because', "'", 'is', 'a', 'word', 'to', 'the', 'whys', '.']
["'", 'Because', "'", 'is', 'a', 'word', 'to', 'the', 'why', 's', '.']
whys
8
why
8
['I', 'have', 'always', 'wanted', 'to', 'hand', 'out', 'carts', 'at', 'Wal', '-', 'Mart', '.', 'I', 'cannot', 'imagine', 'a', 'greeter', 'job', '.']
['I', 'have', 'always', 'wanted', 'to', 'hand', 'out', 'carts', 'at', 'Wal', '-', 'Mart', '.', 'I', 'can', 'not', 'imagine', 'a', 'greeter', 'job', '.']
greeter
17
greeter
18


In [ ]:
# assert len(new_puns_hom) == len(gold_hom)
assert len(gold_hom) == len(new_location_hom)
# assert len(new_puns_het) == len(gold_het)
assert len(gold_het) == len(new_location_het)

In [ ]:
print(len(gold_hom))
print(len(gold_het))
print()
print(sum(gold_hom))
print(sum(gold_het))
print()

num_pos_task_7 = sum(gold_hom + gold_het)
print(num_pos_task_7)
num_neg_task_7 = len(gold_hom + gold_het) - num_pos_task_7
print(num_neg_task_7)
num_delta = num_pos_task_7 - num_neg_task_7
print(num_delta)

# PTD: from iindex=2423 is neg
total_puns = puns_hom + puns_het + texts_PTD[-num_delta:]
total_gold = gold_hom + gold_het + labels_PTD[-num_delta:]
total_location = new_location_hom + new_location_het + [-10000 for _ in range(num_delta)]
# -10000 means has no pun in the sentence
assert len(total_puns) == len(total_gold)
assert len(total_gold) == len(total_location)
assert sum(total_gold) * 2 == len(total_puns)
print()

print(len(total_puns))
print(sum(total_gold))
print(sum([1 for i in total_location if i == -10000]))

2250
1780

1607
1271

2878
1152
1726

5756
2878
2878


In [ ]:
hom_pos_index_list = [i for i, x in enumerate(gold_hom) if x == 1]
het_pos_index_list = [i for i, x in enumerate(gold_het) if x == 1]
print(hom_pos_index_list)
print(het_pos_index_list)

[0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 13, 14, 15, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 32, 33, 37, 38, 40, 42, 43, 44, 45, 46, 47, 49, 50, 51, 52, 54, 57, 59, 60, 61, 62, 66, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 79, 80, 81, 82, 84, 85, 86, 87, 89, 90, 91, 92, 95, 96, 97, 98, 99, 102, 103, 105, 106, 107, 108, 109, 111, 112, 114, 115, 116, 118, 119, 123, 124, 125, 126, 127, 129, 131, 132, 134, 135, 136, 137, 138, 139, 140, 141, 143, 144, 146, 147, 148, 149, 150, 151, 152, 153, 155, 156, 158, 159, 160, 161, 162, 163, 164, 166, 168, 169, 171, 172, 173, 174, 176, 177, 178, 180, 182, 184, 185, 187, 188, 190, 192, 194, 195, 196, 197, 198, 200, 201, 202, 203, 205, 206, 207, 208, 213, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 232, 233, 234, 235, 236, 237, 239, 242, 244, 245, 246, 249, 250, 253, 254, 255, 257, 258, 259, 260, 261, 264, 265, 266, 267, 268, 269, 271, 272, 273, 275, 276, 277, 278, 280, 281, 282, 283, 284, 285, 286, 287, 288, 291, 292,

In [ ]:
def Wu_Palmer_similarity(sentence, word_list):
    """
    Wu-Palmer similarity (wup_similarity):
    Range: [0, 1]
    Minimum: 0 (no relationship)
    Maximum: 1 (identical synsets)
    """
    # No self-loop
    # Example sentence
    # sentence = "The cat chased the dog"

    # Calculate semantic relatedness using Wu-Palmer similarity
    # word1 = 'dog'
    # word2 = 'cat'
    # synset1 = lesk(nltk.word_tokenize(sentence), word1, 'n')  # Disambiguate word1
    # synset2 = lesk(nltk.word_tokenize(sentence), word2, 'n')  # Disambiguate word2

    edge_weight = torch.rand((len(word_list) * (len(word_list) - 1),))
    # Returns a tensor filled with random numbers from a uniform distribution on the interval [0, 1)
    nltk_word_tokenize_sentence = nltk.word_tokenize(sentence)
    synset_list = []

    for word in word_list:
        synset_list.append(lesk(nltk_word_tokenize_sentence, word, 'n'))

    for i in range(len(word_list)):
        for j in range(i + 1, len(word_list)):
            if isinstance(synset_list[0], Synset) and isinstance(synset_list[1], Synset):
                similarity = synset_list[0].wup_similarity(synset_list[1])
                # similarity = synset1.wup_similarity(synset2)
                # The same as synset2.wup_similarity(synset1)
                # print("Similarity between", word1, "and", word2, ":", similarity)
                edge_weight[j * (len(word_list) - 1) + i + (-1 if i > j else 0)] = similarity
                edge_weight[i * (len(word_list) - 1) + j + (-1 if j > i else 0)] = similarity
            #else:
                # print("Cannot compute similarity. One or both words are not in WordNet.")
                # random samples from a uniform distribution over [0, 1).

    return edge_weight

In [ ]:
def create_dataset(texts, labels, tokenizer, max_length, total_location):
    # Each element of the texts is word list
    assert len(texts) == len(labels)
    dataset = []

    for data_index in tqdm(range(len(labels))):
        text, label, location = texts[data_index], labels[data_index], total_location[data_index]
        # Define the input text
        # text = ['We’ve', 'got', 'children’s', 'church', 'because', 'we', 'know', 'how', 'to', 'kid', 'around']
        corresponding_location = -10000
        id_range = [] # The word belong to which ids [start, end)

        # Process the text with the spaCy NLP pipeline
        words = [str(tok) for sent in (nlp(' '.join(text))).sents for tok in sent]
        #words = text
        #print(words)

        corresponding_index = []
        # consider which token (in tokens[], base on index) to be the corresponding word

        current_index = 0
        start = False
        idx = 0
        #print(words)
        for word_idx, x in enumerate(words):
            #print(x)
            subwords = tokenizer.tokenize((' ' if start else '') + x)
            start = True
            #print(subwords)
            for i in range(1, len(subwords)):
                subwords[i] = subwords[i][2:] # delete the '##'

            # subwords_len = np.zeros((len(subwords),), dtype=float)
            subwords_len = torch.zeros((len(subwords),))
            for count_index, each_token in enumerate(subwords):
                subwords_len[count_index] = len(each_token)
            # longest_index = np.argmax(subwords_len, axis=0)
            longest_index = int(torch.argmax(subwords_len, dim=0))

            #print(longest_index)
            if 1 + current_index + longest_index < max_length - 1:
                # Cannot longer than max_length
                # because the input_ids has a start ID and end ID
                # which needs to +1 and -1 to match the index
                corresponding_index.append(current_index + longest_index)
                id_range.append([current_index, current_index + len(subwords)])
                if word_idx == location:
                    corresponding_location = current_index + longest_index

                current_index += len(subwords)
                idx += 1
            else:
                break

        if label > 0.5 and corresponding_location < 0:
            print("Do not get the corresponding_location!")

        original_idx = idx
        if text[:idx] != words[:idx]:
            for each_word in text[:idx]:
                original_idx -= 1
                sub_sent = nlp(' '.join(text[:original_idx]))
                if [str(tok) for sent in sub_sent.sents for tok in sent] == words[:idx]:
                    break
            else:
                print("Something Wrong!")

        # TODO: Some bugs in here, if the cut word is the last word, it maybe wrong
        
        doc = nlp(' '.join(text[:original_idx]))
        assert [str(tok) for sent in doc.sents for tok in sent] == words[:idx]
        graph = nx.DiGraph()
        for token in doc:
            graph.add_node(token.i, word=token.text)
            graph.add_edge(token.head.i, token.i, relation=token.dep_)

        #for node in graph.nodes(data=True):
        #    print(f"Node: {node}")

        #print(corresponding_index)
        #print(words[:idx])
        #print(id_range)
        corresponding_index = torch.tensor(corresponding_index, dtype=torch.long)
        #print(corresponding_index)
        assert len(corresponding_index) == len(words[:idx])
        assert len(corresponding_index) == len(id_range)

        sentence = ' '.join(words) # be processed in tokenizer
        #tokens = tokenizer.tokenize(sentence)
        #print(tokens)
        
        tokenized = tokenizer(sentence, return_tensors="pt", max_length=max_length, return_token_type_ids=False, truncation=True, padding="max_length")
        #print(tokenized)

        num_nodes = len(corresponding_index)
        source_nodes = [i for j in range(num_nodes) for i in range(num_nodes) if i != j]
        target_nodes = [j for j in range(num_nodes) for i in range(num_nodes) if i != j]
        # Complete directed graph, no self-loop

        # Add the root, which is self-loop, let the temporary index is -1, -1, and it is the last element
        source_nodes.append(-1)
        target_nodes.append(-1)

        source_nodes = torch.tensor(source_nodes, dtype=torch.long)
        target_nodes = torch.tensor(target_nodes, dtype=torch.long)
        edge_index = torch.stack([source_nodes, target_nodes], dim=0)
        #print(edge_index)
        # print(edge_index.shape[1]) # the number of edges

        # edge_weight = torch.rand(edge_index.shape[1])  # Replace this with the actual edge weights of your adjacency matrix
        edge_weight = Wu_Palmer_similarity(sentence, words[:idx]) # numpy array
        # edge_weight = np.append(edge_weight, [10.0], axis=0) # add the ROOT value
        edge_weight = torch.cat((edge_weight, torch.tensor([10.0])), dim=-1)
        # edge_weight = torch.tensor(edge_weight, dtype=torch.float32) # the type must dtype=torch.float32
        assert edge_index.shape[1] == len(edge_weight)

        has_ROOT = False
        # Add dependency parsing tree
        for edge in graph.edges(data=True):
            #print(f"Edge: {edge}")
            source_index = edge[0]
            target_index = edge[1]
            if source_index == target_index:
                # ROOT
                source_nodes[-1] = source_index
                target_nodes[-1] = target_index
                edge_index[0, len(source_nodes) - 1] = source_index
                edge_index[1, len(target_nodes) - 1] = target_index
                edge_weight[len(edge_weight) - 1] = 20.0 # may change this number
                # print(edge_index[:, len(edge_weight) - 1])
                has_ROOT = True
            else:
                # not ROOT
                edge_weight[target_index * (num_nodes - 1) + source_index + (-1 if source_index > target_index else 0)] += 0.8
                edge_weight[target_index * (num_nodes - 1) + source_index + (-1 if source_index > target_index else 0)] *= 10
                # may change this number
                # print(edge_index[:, target_index * (num_nodes - 1) + source_index + (-1 if source_index > target_index else 0)])
        if not has_ROOT:
            print("No ROOT, something wrong!")

        y = torch.tensor(label, dtype=torch.long)

        #tokenized["input_ids"][0] == tokenized["input_ids"].flatten()
        #print(corresponding_index)
        if len(tokenized["input_ids"][0]) != max_length:
            print(len(tokenized["input_ids"][0]))
            print("The length is wrong!")
        for each_corre_idx, each_corre in enumerate(corresponding_index):
            assert id_range[each_corre_idx][0] <= each_corre < id_range[each_corre_idx][1]

        data = Data(data_identification=torch.tensor(data_index, dtype=torch.long), # The ID of the data
                    input_ids=tokenized["input_ids"][0],
                    attention_mask=tokenized["attention_mask"][0],
                    corresponding=corresponding_index,
                    sentence_len=torch.tensor(len(corresponding_index), dtype=torch.long), # original length
                    edge_index=edge_index,
                    edge_weight=edge_weight,
                    location=torch.tensor(location, dtype=torch.long), # pun word index in original sentence
                    corresponding_location=torch.tensor(corresponding_location, dtype=torch.long), # pun word index in ids (start with 0), need to +1
                    id_range=torch.tensor(id_range, dtype=torch.long),
                    y=y,
                    num_nodes=torch.tensor(num_nodes, dtype=torch.long)
                    )
        # DO NOT corresponding_index=corresponding_index!
        # If the end of the name of parameters of Data() has '_index',
        # the package will consider this is index list, it will be automatic update the value list:
        # Example: [1,2,5], [2,3,9], [3,6,1] -> [1,2,5,8,9,15,19,22,17]
        # Not: [1,2,5], [2,3,9], [3,6,1] -> [1,2,5,2,3,9,3,6,1]
        # If set num_nodes as parameters of Data(), it will be sum automatically (is a number finally, not a list)
        dataset.append(data)
        '''
        print('tokenized["input_ids"][0]')
        print(tokenized["input_ids"][0])
        print('tokenized["attention_mask"][0]')
        print(tokenized["attention_mask"][0])
        print('corresponding_index')
        print(corresponding_index)
        print('len(corresponding_index)')
        print(len(corresponding_index))
        print('location')
        print(location)
        print('corresponding_location')
        print(corresponding_location)
        print('id_range')
        print(id_range)
        print('y')
        print(y)
        '''

    return dataset

In [ ]:
# Custom GNN and BiLSTM layers
class GNN(torch.nn.Module):

    def __init__(self, input_dim, hidden_dim, output_dim):
        # input_dim is number of features of each node
        super(GNN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, x, edge_index, edge_weight, batch):
        x = self.conv1(x, edge_index, edge_weight)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index, edge_weight)

        # Global Pooling (stack different aggregations)
        hidden = torch.cat([global_mean_pool(x, batch),
                            global_max_pool(x, batch)], dim=1)
        return x, hidden


class BERT_GNN_MTL_Classifier(nn.Module):
    def __init__(self, num_classes, hidden_dim, num_lstm_layers, gnn_hidden_dim, gnn_output_dim, max_length, batch_size):
        super().__init__()
        self.max_length = max_length
        self.batch_size = batch_size
        self.gnn_output_dim = gnn_output_dim
        self.bert_config = BertConfig.from_pretrained(bert_model_name)
        self.bert = BertModel.from_pretrained(bert_model_name, config=self.bert_config)
        # Embedding size of BERT is 768
        self.gnn = GNN(self.bert_config.hidden_size, gnn_hidden_dim, gnn_output_dim)
        #self.bilstm = BiLSTM(self.bert_config.hidden_size, hidden_dim, num_lstm_layers)
        #self.bilstm = BiLSTM(gnn_output_dim, hidden_dim, num_lstm_layers)
        self.bilstm = nn.LSTM(
            input_size=gnn_output_dim,
            hidden_size=hidden_dim,
            num_layers=num_lstm_layers,
            bidirectional=True,
            batch_first=True,
            #dropout=dropout if num_layers > 1 else 0
        )
        self.fc = nn.Linear(hidden_dim * 2, 1) # 2 is bidirectional LSTM, has 2, 1 is the output
        # self.fc = nn.Linear(hidden_dim * 2, num_classes) # multi-class classfication
        # self.sigmoid = nn.Sigmoid()
        self.classifier = nn.Linear(hidden_dim * (2 if self.bilstm.bidirectional else 1), 1) # sigmoid

    def forward(self, data, token_index):
        # token_index[i] should be start with 0 and step is 1, because the GNN use only one id per word
        input_ids, attention_mask = data.input_ids, data.attention_mask
        corresponding_index, sentence_len = data.corresponding, data.sentence_len
        edge_index, edge_weight, num_nodes, batch = data.edge_index, data.edge_weight, data.num_nodes, data.batch
        #print(sentence_len)
        assert sum(sentence_len) == len(corresponding_index)
        actual_batch_size = len(input_ids) // self.max_length

        first_cut = 0
        corres = [] # the corresponding index of each sentence
        # print(corresponding_index)
        # print(sentence_len)
        for each_sent_len_i in range(len(sentence_len) - 1):
            corres.append(corresponding_index[first_cut:first_cut+sentence_len[each_sent_len_i]])
            first_cut += sentence_len[each_sent_len_i]
        corres.append(corresponding_index[first_cut:first_cut+sentence_len[-1]])
        # print(corres)
        assert len(corres) == actual_batch_size

        input_ids = torch.reshape(input_ids, (actual_batch_size, self.max_length))
        attention_mask = torch.reshape(attention_mask, (actual_batch_size, self.max_length))
        # print("input_ids.shape:", input_ids.shape) # torch.Size([self.batch_size, self.max_length])

        bert_output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        hidden_states = bert_output.last_hidden_state
        # print("hidden_states.shape:", hidden_states.shape) # torch.Size([self.batch_size, self.max_length, 768])

        gnn_input = torch.zeros((len(corresponding_index), hidden_states.shape[-1])) # not include the start and end ID
        each_batch_index = 0
        start_index = 0
        each_counter = 0
        # print(corresponding_index)
        for each_corr_i in corresponding_index:
            # each_corr_i + 1 because the input_ids has a start ID in the front,
            # which needs to add 1 to match the index
            gnn_input[start_index] = hidden_states[each_batch_index, each_corr_i + 1]
            start_index += 1
            each_counter += 1
            if each_counter == sentence_len[each_batch_index]:
                each_batch_index += 1
                each_counter = 0
        each_counter = 0
        assert each_batch_index == actual_batch_size
        assert start_index == len(corresponding_index)
        
        gnn_output, gnn_hidden = self.gnn(x=gnn_input, edge_index=edge_index, edge_weight=edge_weight, batch=batch)

        bilstm_input = torch.zeros((actual_batch_size, self.max_length, self.gnn_output_dim))
        each_batch_index = 0
        start_index = 0
        # Similar to padding 0
        for each_sen_len in sentence_len:
            for each_word_i in range(each_sen_len):
                bilstm_input[each_batch_index, each_word_i] = gnn_output[start_index]
                start_index += 1
            each_batch_index += 1

        each_batch_index = 0
        start_index = 0

        lstm_output, (hidden, _) = self.bilstm(bilstm_input)
        '''
        Anthor way is using only last hidden state of the LSTM cell:

        Using lstm_output[:, -1] selects the last hidden state of the LSTM cell
        for each sequence in the batch. The reason we use this approach in the
        example provided is that the last hidden state is often a good
        representation of the entire sequence in many sequence-to-sequence
        models, especially for classification tasks.

        When we use lstm_output[:, -1], we're selecting the hidden states of the
        LSTM cells at the last time step (i.e., the last token in the input sequence)
        for each sequence in the batch. This can be a good representation of the
        entire sequence for classification tasks since it captures information
        from both the forward and backward passes of the sequence.
        '''
        pooled_output = torch.mean(lstm_output, 1)

        if self.bilstm.bidirectional:
            hidden = torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1)
            # hidden = torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=-1)
        else:
            hidden = hidden[-1, :, :]

        # Get the hidden state of the word at the specified index
        assert lstm_output.size(0) == len(input_ids)
        assert actual_batch_size == lstm_output.size(0)
        assert lstm_output.size(2) == self.bilstm.hidden_size * (2 if self.bilstm.bidirectional else 1)

        focused_word_hidden = torch.zeros((lstm_output.size(0), lstm_output.size(2)))
        for sentence_index in range(lstm_output.size(0)):
            # token_index[i] should be start with 0 and step is 1, because the GNN use only one id per word
            # If is the pun, token_index[i] should be the same as data.location
            # corresponding index (need to be +1) of the pun word: int(corres[sentence_index][token_index[sentence_index]])
            # print(int(corres[sentence_index][token_index[sentence_index]]))
            focused_word_hidden[sentence_index] = lstm_output[sentence_index, token_index[sentence_index], :]
        classification_output = self.classifier(focused_word_hidden)

        return self.fc(pooled_output), classification_output
        

In [ ]:
# **sigmoid**
# If using sigmoid, we should use this accuracy function
def calculate_accuracy(output, target):
    assert output.shape == target.shape
    threshold = 0.5
    predictions = (output > threshold).float()
    correct = (predictions == target).sum().item()
    total = target.numel()
    return correct / total

def train_epoch(model, data_loader, loss_fn_1, loss_fn_2, optimizer, device):
    model.train()
    total_loss = 0.0
    total_accuracy_1 = 0.0
    total_accuracy_2 = 0.0
    for batch in tqdm(data_loader):
        batch = batch.to(device)
        labels = batch.y.unsqueeze(1).float().to(device)

        token_index = torch.clone(batch.location).to(device)

        batch_att = torch.reshape(batch.attention_mask, (len(labels), len(batch.attention_mask) // len(labels)))

        for sentence_index in range(len(labels)):
            if token_index[sentence_index] < 0:
                # No pun word
                assert labels[sentence_index][0] < 0.5
                token_index[sentence_index] = torch.randint(0, int(sum(batch_att[sentence_index])) - 2, (1,))
                # TODO: Should be modified to sentence_len
        # print(token_index)
        # print(batch.id_range)

        optimizer.zero_grad()
        logits_1, logits_2 = model(batch, token_index)
        loss_1 = loss_fn_1(logits_1, labels)
        loss_2 = loss_fn_2(logits_2, labels)
        joint_loss = loss_1 + loss_2
        joint_loss.backward()
        optimizer.step()

        total_loss += joint_loss.item()
        sigmoid_1 = torch.sigmoid(logits_1.view(-1)).unsqueeze(1)
        sigmoid_2 = torch.sigmoid(logits_2.view(-1)).unsqueeze(1)
        total_accuracy_1 += calculate_accuracy(sigmoid_1, labels)
        total_accuracy_2 += calculate_accuracy(sigmoid_2, labels)
        #print(f"Train Loss: {total_loss:.4f}, Train Accuracy: {total_accuracy:.4f}")

    return total_loss / len(data_loader), total_accuracy_1 / len(data_loader), total_accuracy_2 / len(data_loader)


def eval_epoch(model, data_loader, loss_fn_1, loss_fn_2, device):
    model.eval()
    total_loss = 0.0
    total_accuracy_1 = 0.0
    total_accuracy_2 = 0.0
    with torch.no_grad():
        for batch in tqdm(data_loader):
            batch = batch.to(device)
            labels = batch.y.unsqueeze(1).float().to(device)

            token_index = torch.clone(batch.location).to(device)

            batch_att = torch.reshape(batch.attention_mask, (len(labels), len(batch.attention_mask) // len(labels)))

            for sentence_index in range(len(labels)):
                if token_index[sentence_index] < 0:
                    # No pun word
                    assert labels[sentence_index][0] < 0.5
                    token_index[sentence_index] = torch.randint(0, int(sum(batch_att[sentence_index])) - 2, (1,))
                    # TODO: Should be modified to sentence_len
                    # random select one location

            logits_1, logits_2 = model(batch, token_index)
            loss_1 = loss_fn_1(logits_1, labels)
            loss_2 = loss_fn_2(logits_2, labels)
            joint_loss = loss_1 + loss_2

            total_loss += joint_loss.item()
            sigmoid_1 = torch.sigmoid(logits_1.view(-1)).unsqueeze(1)
            sigmoid_2 = torch.sigmoid(logits_2.view(-1)).unsqueeze(1)
            total_accuracy_1 += calculate_accuracy(sigmoid_1, labels)
            total_accuracy_2 += calculate_accuracy(sigmoid_2, labels)
            #print(f"Validation Loss: {total_loss:.4f}, Validation Accuracy: {total_accuracy:.4f}")

    return total_loss / len(data_loader), total_accuracy_1 / len(data_loader), total_accuracy_2 / len(data_loader)

In [ ]:
# Assuming you have your data as lists: train_texts, train_labels, val_texts, and val_labels
tokenizer = BertTokenizer.from_pretrained(bert_model_name)
max_length = 80  # Adjust the maximum length based on your dataset
batch_size = 32

In [ ]:
# Create data loaders
dataset = create_dataset(texts=total_puns, labels=total_gold,
                         tokenizer=tokenizer, max_length=max_length,
                         total_location=total_location)

fileObj = open('/content/drive/My Drive/dataset.obj', 'wb')
pickle.dump(dataset, fileObj)
fileObj.close()

fileObj = open('/content/drive/My Drive/dataset.obj', 'rb')
dataset = pickle.load(fileObj)
fileObj.close()

# Split the dataset into training and validation sets
train_dataset, val_dataset = torch.utils.data.random_split(
    dataset, [len(total_gold)-int(0.2*len(total_gold)), int(0.2*len(total_gold))])

fileObj = open('/content/drive/My Drive/train_dataset.obj', 'wb')
pickle.dump(train_dataset, fileObj)
fileObj.close()

fileObj = open('/content/drive/My Drive/train_dataset.obj', 'rb')
train_dataset = pickle.load(fileObj)
fileObj.close()

fileObj = open('/content/drive/My Drive/val_dataset.obj', 'wb')
pickle.dump(val_dataset, fileObj)
fileObj.close()

fileObj = open('/content/drive/My Drive/val_dataset.obj', 'rb')
val_dataset = pickle.load(fileObj)
fileObj.close()

# Create DataLoaders for each set with a batch size
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

100%|██████████| 5756/5756 [03:58<00:00, 24.14it/s]
/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [ ]:
fileObj = open('/content/drive/My Drive/dataset.obj', 'rb')
dataset = pickle.load(fileObj)
fileObj.close()

fileObj = open('/content/drive/My Drive/train_dataset.obj', 'rb')
train_dataset = pickle.load(fileObj)
fileObj.close()

fileObj = open('/content/drive/My Drive/val_dataset.obj', 'rb')
val_dataset = pickle.load(fileObj)
fileObj.close()

In [ ]:
# Initialize the custom BERT model
# num_classes = len(set(total_gold))  # Assuming labels are integers starting from 0
num_classes = 2
hidden_dim = 128
num_lstm_layers = 2
gnn_hidden_dim = 512
gnn_output_dim = 256
learning_rate = 2e-5


# Set up two different loss function
loss_fn_1 = nn.BCEWithLogitsLoss()
loss_fn_2 = nn.BCEWithLogitsLoss()

In [ ]:
# Build the model
model = BERT_GNN_MTL_Classifier(num_classes, hidden_dim, num_lstm_layers,
                                gnn_hidden_dim, gnn_output_dim, max_length, batch_size).to(device)
print(model)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BERT_GNN_MTL_Classifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, eleme

In [ ]:
# Train the model
num_epochs = 5

for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    train_loss, train_acc_1, train_acc_2 = train_epoch(model, train_loader, loss_fn_1, loss_fn_2, optimizer, device)
    print(f"Training Loss: {train_loss:.4f}, Training Accuracy: {train_acc_1:.4f}, Training Word Accuracy: {train_acc_2:.4f}")
    torch.save(model, model_path) # save the entire model, including the architecture
    val_loss, val_acc_1, val_acc_2 = eval_epoch(model, val_loader, loss_fn_1, loss_fn_2, device)
    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc_1:.4f}, Validation Word Accuracy: {val_acc_2:.4f}")
print("Training complete.")

Epoch 1/5


100%|██████████| 144/144 [1:08:24<00:00, 28.50s/it]


Training Loss: 1.0905, Training Accuracy: 0.7912, Training Word Accuracy: 0.8748


100%|██████████| 36/36 [05:11<00:00,  8.64s/it]


Validation Loss: 0.8388, Validation Accuracy: 0.9209, Validation Word Accuracy: 0.9217
Epoch 2/5


...

100%|██████████| 144/144 [1:06:08<00:00, 27.56s/it]


Training Loss: 0.4844, Training Accuracy: 0.9668, Training Word Accuracy: 0.9670


100%|██████████| 36/36 [05:10<00:00,  8.61s/it]


Validation Loss: 0.5063, Validation Accuracy: 0.9391, Validation Word Accuracy: 0.9435
Epoch 4/5


  4%|▍         | 6/144 [02:49<1:03:45, 27.72s/it]

In [ ]:
# Load the the saved file
loaded_model = torch.load(model_path).to(device)

# Set the model to evaluation mode if you plan to use it for inference
loaded_model.eval()

location_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)

with torch.no_grad():

    y_true_hom = []
    y_pred_hom = []
    y_true_het = []
    y_pred_het = []
    y_true_neg = []
    y_pred_neg = []

    location_hom_correct = 0
    location_het_correct = 0

    num_correct, num_samples, num_accuracy = 0, 0, 0
    for ele in tqdm(location_loader):
        # print(ele.attention_mask)
        attention_mask = ele.attention_mask.reshape(1, max_length).to(device)
        #labels = ele['label'].unsqueeze(1).float().to(device)
        label = ele.y.to(device)
        # corresponding_token_index = torch.clone(ele.corresponding_location).to(device)
        token_index = torch.clone(ele.location).to(device) # After GNN, the same as original, start with 0
        # id_range = ele.id_range.to(device)
        data_identification = ele.data_identification.to(device)

        #print(input_ids)
        #print(attention_mask)
        #print(label[0])
        #print(int(token_index[0]))
        #print(id_range)
        #print(ele)

        if float(label[0]) < 0.5:
            y_true_neg.append(0)
            # This has not pun, do not count
            logits_1, _ = loaded_model(ele, torch.tensor(0).reshape(1,))
            y_pred_neg.append(0 if float(torch.sigmoid(logits_1.view(-1))) < 0.5 else 1)
            continue
        
        scores_list = torch.zeros((int(sum(attention_mask[0])) - 2,))
        for used_token_index in range(int(sum(attention_mask[0])) - 2): # TODO: Should be modified to sentence_len
            # After GNN, the index is same as original, start with 0
            logits_1, logits_2 = loaded_model(ele, torch.tensor(used_token_index).reshape(1,))
            scores_list[used_token_index] = float(torch.sigmoid(logits_2.view(-1))) # Get each ID scores

        # TODO: If the matched token ID is belonged to the pun word, it is also correct.
        if int(torch.argmax(scores_list, dim=0)) == int(token_index[0]):
            num_correct += 1

            if int(data_identification[0]) < 2250:
                location_hom_correct += 1
            elif 2250 <= int(data_identification[0]) < 2250 + 1780:
                location_het_correct += 1
            else:
                print("Something Wrong")
        
        if int(data_identification[0]) < 2250:
            y_true_hom.append(1)
            y_pred_hom.append(0 if float(torch.sigmoid(logits_1.view(-1))) < 0.5 else 1)
        elif 2250 <= int(data_identification[0]) < 2250 + 1780:
            y_true_het.append(1)
            y_pred_het.append(0 if float(torch.sigmoid(logits_1.view(-1))) < 0.5 else 1)
        else:
            print("Something Wrong")

        num_samples += 1
        #if num_samples == 300:
        #    break

    num_accuracy = num_correct / num_samples
    print(f'Location Accuracy: {num_accuracy:.4f}')
    print(f'Homographic Location Accuracy: {(location_hom_correct / len(y_true_hom)):.4f}')
    print(f'Heterographic Location Accuracy: {(location_het_correct / len(y_true_het)):.4f}')
    assert len(y_true_hom) + len(y_true_het) == num_samples
    print()
    print(classification_report(y_true_hom+y_true_neg[:len(y_true_hom)], y_pred_hom+y_pred_neg[:len(y_true_hom)], target_names=['No Puns', 'Homographic']))
    print(confusion_matrix(y_true_hom+y_true_neg[:len(y_true_hom)], y_pred_hom+y_pred_neg[:len(y_true_hom)]))
    print()
    print(classification_report(y_true_het+y_true_neg[:len(y_true_het)], y_pred_het+y_pred_neg[:len(y_true_het)], target_names=['No Puns', 'Heterographic']))
    print(confusion_matrix(y_true_het+y_true_neg[:len(y_true_het)], y_pred_het+y_pred_neg[:len(y_true_het)]))

100%|██████████| 1151/1151 [53:58<00:00,  2.81s/it]

Location Accuracy: 0.0564
Homographic Location Accuracy: 0.0583
Heterographic Location Accuracy: 0.0543

              precision    recall  f1-score   support

     No Puns       0.95      0.94      0.94       309
 Homographic       0.94      0.95      0.94       309

    accuracy                           0.94       618
   macro avg       0.94      0.94      0.94       618
weighted avg       0.94      0.94      0.94       618

[[290  19]
 [ 16 293]]

               precision    recall  f1-score   support

      No Puns       0.94      0.93      0.94       258
Heterographic       0.93      0.94      0.94       258

     accuracy                           0.94       516
    macro avg       0.94      0.94      0.94       516
 weighted avg       0.94      0.94      0.94       516

[[241  17]
 [ 15 243]]
